In [40]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt

from surprise import Dataset, Reader, accuracy
from surprise.model_selection import cross_validate, train_test_split, GridSearchCV
from surprise.prediction_algorithms import SVD, SVDpp, NMF, BaselineOnly, NormalPredictor

%matplotlib inline

In [41]:
!ls Data

meta_Beauty.json.gz      reviews_Beauty_5.json.gz


In [42]:
df = pd.read_json("Data/reviews_Beauty_5.json.gz",lines=True)

In [5]:
df

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1YJEY40YUW4SE,7806397051,Andrea,"[3, 4]",Very oily and creamy. Not at all what I expect...,1,Don't waste your money,1391040000,"01 30, 2014"
1,A60XNB876KYML,7806397051,Jessica H.,"[1, 1]",This palette was a decent price and I was look...,3,OK Palette!,1397779200,"04 18, 2014"
2,A3G6XNM240RMWA,7806397051,Karen,"[0, 1]",The texture of this concealer pallet is fantas...,4,great quality,1378425600,"09 6, 2013"
3,A1PQFP6SAJ6D80,7806397051,Norah,"[2, 2]",I really can't tell what exactly this thing is...,2,Do not work on my face,1386460800,"12 8, 2013"
4,A38FVHZTNQ271F,7806397051,Nova Amor,"[0, 0]","It was a little smaller than I expected, but t...",3,It's okay.,1382140800,"10 19, 2013"
...,...,...,...,...,...,...,...,...,...
198497,A2BLFCOPSMBOZ9,B00LLPT4HI,Dave Edmiston,"[0, 0]",Just a little dab of this shea butter should b...,5,A little dab...,1405468800,"07 16, 2014"
198498,A1UQBFCERIP7VJ,B00LLPT4HI,Margaret Picky,"[0, 0]",This shea butter is completely raw and unrefin...,5,Pure organic raw shea butter,1405296000,"07 14, 2014"
198499,A35Q0RBM3YNQNF,B00LLPT4HI,M. Hill,"[0, 0]",The skin is the body's largest organ and it ab...,5,One Pound Organic Grade A Unrefined Shea Butter,1405468800,"07 16, 2014"
198500,A3LGT6UZL99IW1,B00LLPT4HI,"Richard C. Drew ""Anaal Nathra/Uthe vas Bethod...","[0, 0]",I have very dry elbows and knees. I have a to...,5,This stuff is amazing!,1405382400,"07 15, 2014"


In [6]:
df.isna().sum()

reviewerID           0
asin                 0
reviewerName      1386
helpful              0
reviewText           0
overall              0
summary              0
unixReviewTime       0
reviewTime           0
dtype: int64

In [7]:
#1386 reviewerNames left blank; we will not need reviewerName since utilizing reviewerID but 
#curious why

In [8]:
df[df['reviewerName'].isnull()]

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
8,A3LMILRM9OC3SA,9759091062,NaN,"[0, 0]",Did nothing for me. Stings when I put it on. I...,2,"no Lightening, no Brightening,......NOTHING",1405209600,"07 13, 2014"
1790,AK1H26O8DLMNN,B0000535UM,NaN,"[0, 0]",The first thickening shampoo that works on my ...,5,Actually works,1405123200,"07 12, 2014"
2242,APTLHR9PHGPXN,B00005NAOD,NaN,"[0, 0]","Kind of drying, not moisturizing. Kind of disa...",2,Inexpensive and feels that wY,1405209600,"07 13, 2014"
2304,AQWX644AFUFFK,B00005NFBD,NaN,"[0, 0]","This is just ok. For one, I found this in a st...",3,"Ok, nothing amazing",1405468800,"07 16, 2014"
3651,A43K5ZRQ87TO6,B00008PC1O,NaN,"[0, 0]",Works well and easy to use!,5,Five Stars,1405296000,"07 14, 2014"
...,...,...,...,...,...,...,...,...,...
197192,A1Z3AV93ONK5VF,B00KAL5JAU,NaN,"[0, 0]",We already had the Dead Sea Shampoo by Adovia ...,5,"Non greasy, silky shiny hair",1401235200,"05 28, 2014"
197193,A184I8GT3BHZQV,B00KAL5JAU,NaN,"[0, 1]",&#60;a href=&#34;http://www.tomoson.com/?code=...,5,Dead Sea Salt hair conditioner,1403568000,"06 24, 2014"
197194,A8C9EJORQD23,B00KAL5JAU,NaN,"[0, 1]",I use this with the Adovia shampoo I mention a...,5,Adovia does natural right!,1402272000,"06 9, 2014"
198446,A2PIGZCDGM4NJ7,B00L5JHZJO,NaN,"[10, 11]","This is a horrible product, most of the review...",1,Bad Product,1404864000,"07 9, 2014"


In [9]:
#all necessary information is included even when reviewerName is NaN

In [10]:
df.reviewerID.value_counts()

A2V5R832QCSOMX    204
ALNFHVS3SC4FV     192
AKMEY1BSHSDG7     182
A3KEZLJ59C1JVH    154
ALQGOMOY1F5X9     150
                 ... 
A32JLLK5UZN361      5
A2S17E3KLIP3S6      5
A4BVAE9DUWY7A       5
A1NM70Z0LAQGMA      5
A16WEI24KY91IY      5
Name: reviewerID, Length: 22363, dtype: int64

In [11]:
#YAY! all reviewerIDs have value of at least 5, total of 22,363 reviewers

In [12]:
df.asin.value_counts()

B004OHQR1Q    431
B0043OYFKU    403
B0069FDR96    391
B000ZMBSPE    389
B00150LT40    329
             ... 
B001TH1480      5
B004NN6JZA      5
B0009MMK5M      5
B002IACDGY      5
B004YM9SVM      5
Name: asin, Length: 12101, dtype: int64

In [13]:
#YAY! all products have at least 5 reviews, total of 12,101 different products

In [14]:
#can save data as a CSV file for easier use, but it does not look like we need to clean data
#can drop reviewerNames

In [15]:
lower_rating = df.overall.min()

In [16]:
upper_rating = df.overall.max()

In [17]:
print('Review range: {0} to {1}'.format(lower_rating, upper_rating))

Review range: 1 to 5


In [18]:
#Confirming our review range is 1 to 5, which is the default for surprise

In [19]:
#Creating dataframe with appropriate columns to run through surprise

In [44]:
surprise_df = df[['reviewerID', 'asin', 'overall']]

In [45]:
surprise_df

,reviewerID,asin,overall
0,A1YJEY40YUW4SE,7806397051,1
1,A60XNB876KYML,7806397051,3
2,A3G6XNM240RMWA,7806397051,4
3,A1PQFP6SAJ6D80,7806397051,2
4,A38FVHZTNQ271F,7806397051,3
...,...,...,...
198497,A2BLFCOPSMBOZ9,B00LLPT4HI,5
198498,A1UQBFCERIP7VJ,B00LLPT4HI,5
198499,A35Q0RBM3YNQNF,B00LLPT4HI,5
198500,A3LGT6UZL99IW1,B00LLPT4HI,5


In [46]:
reader = Reader(rating_scale=(1, 5))
surprise_data = Dataset.load_from_df(surprise_df, reader)

trainset, testset = train_test_split(surprise_data, test_size=0.2, random_state=23)

In [47]:
surprise_data

In [48]:
# How many users and items are in the trainset
print('Number of users: ', trainset.n_users, '\n')
print('Number of items: ', trainset.n_items, '\n')

Number of users:  22359 

Number of items:  12101 



In [25]:
#10,000 more users than items

In [26]:
print('Type trainset :',type(trainset),'\n')
print('Type testset :',type(testset))

Type trainset : <class 'surprise.trainset.Trainset'> 

Type testset : <class 'list'>


In [27]:
print(len(testset))
print(testset[0])

39701
('A32POM4ALTYIZV', 'B005Z4QT7E', 2.0)


In [28]:
print(len(df))

198502


In [29]:
198502 * .2

39700.4

In [30]:
#sanity check that test set is 20% of total data

In [31]:
#starting with item-item similarity

In [32]:
sim_cos = {'name':'cosine', 'user_based':False}

In [33]:
from surprise.prediction_algorithms import knns

In [34]:
basic = knns.KNNBasic(sim_options=sim_cos)
basic.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [35]:
# looking at the similarity metrics of each of the items to one another by using the sim attribute of our fitted model

In [36]:
basic.sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [37]:
predictions = basic.test(testset)

In [38]:
print(accuracy.rmse(predictions))

RMSE: 1.2129
1.2128933192100995


In [39]:
#RMSE of about 1.2, meaning that it was off by roughly 1.2 points for each guess it made for ratings

In [40]:
sim_cos_user = {'name':'cosine', 'user_based':True}

In [41]:
basic_user = knns.KNNBasic(sim_options=sim_cos_user)
basic_user.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [42]:
basic_user.sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [43]:
predictions_user = basic_user.test(testset)

In [44]:
print(accuracy.rmse(predictions_user))

RMSE: 1.2401
1.2400588428109054


In [45]:
#KNN model with Pearson Correlation 

In [46]:
sim_pearson = {'name':'pearson', 'user_based':False}
basic_pearson = knns.KNNBasic(sim_options=sim_pearson)
basic_pearson.fit(trainset)
predictions = basic_pearson.test(testset)
print(accuracy.rmse(predictions))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 1.1854
1.1854039491870065


In [47]:
sim_pearson_user = {'name':'pearson', 'user_based':True}
basic_pearson_user = knns.KNNBasic(sim_options=sim_pearson_user)
basic_pearson_user.fit(trainset)
predictions_user = basic_pearson_user.test(testset)
print(accuracy.rmse(predictions_user))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 1.1795
1.1795381227887218


In [48]:
#KNN model with Means

In [49]:
#same thing as the basic KNN model, except it takes into account the mean rating of each user or item depending on 
#whether you are performing user-user or item-item similarities, respectively.

In [50]:
sim_pearson = {'name':'pearson', 'user_based':False}
knn_means = knns.KNNWithMeans(sim_options=sim_pearson)
knn_means.fit(trainset)
predictions = knn_means.test(testset)
print(accuracy.rmse(predictions))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 1.1791
1.1790736593254243


In [51]:
sim_pearson_user = {'name':'pearson', 'user_based':True}
knn_means_user = knns.KNNWithMeans(sim_options=sim_pearson_user)
knn_means_user.fit(trainset)
predictions_user = knn_means_user.test(testset)
print(accuracy.rmse(predictions_user))

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 1.1622
1.162184545973618


In [52]:
#This is our best model so far

In [53]:
#Running an SVD model with defaults

In [54]:
svd = SVD(random_state=42)
svd.fit(trainset)
predictions = svd.test(testset)
print(accuracy.rmse(predictions))

RMSE: 1.0889
1.0889451149217502


In [55]:
#Lowest RMSE so far!

In [56]:
svd.predict('A1YJEY40YUW4SE', 'B00LLPT4HI')

Prediction(uid='A1YJEY40YUW4SE', iid='B00LLPT4HI', r_ui=None, est=4.41104541567184, details={'was_impossible': False})

In [57]:
svd.predict('A2BLFCOPSMBOZ9', '7806397051')

Prediction(uid='A2BLFCOPSMBOZ9', iid='7806397051', r_ui=None, est=3.774499861592997, details={'was_impossible': False})

In [58]:
#Checking to see estimated rating for 2 user/product combinations

In [59]:
trainset

In [60]:
cv_svd_baseline = cross_validate(svd, surprise_data)

In [61]:
cv_svd_baseline

{'test_rmse': array([1.08700444, 1.08489981, 1.09317406, 1.09261469, 1.08989769]),
 'test_mae': array([0.83503627, 0.83178077, 0.83852561, 0.83697925, 0.83553279]),
 'fit_time': (13.767770767211914,
  15.25001335144043,
  14.931718111038208,
  13.859838008880615,
  18.23142099380493),
 'test_time': (0.698718786239624,
  0.8477070331573486,
  0.45821189880371094,
  0.6572837829589844,
  1.6967580318450928)}

In [62]:
df

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1YJEY40YUW4SE,7806397051,Andrea,"[3, 4]",Very oily and creamy. Not at all what I expect...,1,Don't waste your money,1391040000,"01 30, 2014"
1,A60XNB876KYML,7806397051,Jessica H.,"[1, 1]",This palette was a decent price and I was look...,3,OK Palette!,1397779200,"04 18, 2014"
2,A3G6XNM240RMWA,7806397051,Karen,"[0, 1]",The texture of this concealer pallet is fantas...,4,great quality,1378425600,"09 6, 2013"
3,A1PQFP6SAJ6D80,7806397051,Norah,"[2, 2]",I really can't tell what exactly this thing is...,2,Do not work on my face,1386460800,"12 8, 2013"
4,A38FVHZTNQ271F,7806397051,Nova Amor,"[0, 0]","It was a little smaller than I expected, but t...",3,It's okay.,1382140800,"10 19, 2013"
...,...,...,...,...,...,...,...,...,...
198497,A2BLFCOPSMBOZ9,B00LLPT4HI,Dave Edmiston,"[0, 0]",Just a little dab of this shea butter should b...,5,A little dab...,1405468800,"07 16, 2014"
198498,A1UQBFCERIP7VJ,B00LLPT4HI,Margaret Picky,"[0, 0]",This shea butter is completely raw and unrefin...,5,Pure organic raw shea butter,1405296000,"07 14, 2014"
198499,A35Q0RBM3YNQNF,B00LLPT4HI,M. Hill,"[0, 0]",The skin is the body's largest organ and it ab...,5,One Pound Organic Grade A Unrefined Shea Butter,1405468800,"07 16, 2014"
198500,A3LGT6UZL99IW1,B00LLPT4HI,"Richard C. Drew ""Anaal Nathra/Uthe vas Bethod...","[0, 0]",I have very dry elbows and knees. I have a to...,5,This stuff is amazing!,1405382400,"07 15, 2014"


In [63]:
#Baseline Models; true baseline should be NormalPredictor(); user_based=True

In [64]:
baseline = NormalPredictor()
baseline.fit(trainset)

In [65]:
predictions = baseline.test(testset)

In [66]:
baseline = accuracy.rmse(predictions)

RMSE: 1.5092


In [67]:
baseline2 = BaselineOnly()
baseline2.fit(trainset)

Estimating biases using als...


In [68]:
predictions2 = baseline2.test(testset)

In [69]:
baseline2 = accuracy.rmse(predictions2)

RMSE: 1.0890


In [71]:
#Gridsearch using NMF 

#Load the algorithm
from surprise.prediction_algorithms import NMF 
from surprise.model_selection import cross_validate, GridSearchCV

#Set NMF grid parameters
#nmf_grid_params = {"biased": [True, False],
                   "n_epochs": [25, 50, 75],
                   "n_factors": [10, 15, 20],
                   
                  }
# instantiate the model

#nmf_gs_model = GridSearchCV(NMF, param_grid=nmf_grid_params, cv=3, return_train_measures=True)

#fit the model
#nmf_gs_model.fit(surprise_data)

#return model with best parameters for rsme 


TypeError: 'dict' object is not callable

In [73]:
#nmf_gs_model.best_params

{'rmse': {'biased': True, 'n_epochs': 25, 'n_factors': 10},
 'mae': {'biased': True, 'n_epochs': 25, 'n_factors': 10}}

In [74]:
nmf_best_params_model = NMF(biased=True, n_epochs=25, n_factors=10)
nmf_best_params_model.fit(trainset)

In [75]:
nmf_best_params_predictions = nmf_best_params_model.test(testset)

In [76]:
accuracy.rmse(nmf_best_params_predictions)

RMSE: 1.1078


1.1077601300622335

In [77]:
#Running with lower params because original best params were the lowest options
#nmf_grid_params_low = {"biased": [True, False],
                   #"n_epochs": [15, 20, 25],
                   #"n_factors": [2, 5, 10]}
#instantiate the model
#nmf_gs_model_low = GridSearchCV(NMF, param_grid=nmf_grid_params_low, cv=3, return_train_measures=True)

#fit the model
#nmf_gs_model_low.fit(surprise_data)

In [78]:
#nmf_gs_model_low.best_params

{'rmse': {'biased': True, 'n_epochs': 15, 'n_factors': 2},
 'mae': {'biased': False, 'n_epochs': 15, 'n_factors': 10}}

In [79]:
nmf_gs_model_low_bp = NMF(biased=True, n_epochs=15, n_factors=2)

In [80]:
nmf_gs_model_low_bp.fit(trainset)

In [81]:
nmf_gs_model_low_predictions = nmf_gs_model_low_bp.test(testset)

In [82]:
accuracy.rmse(nmf_gs_model_low_predictions)

RMSE: 1.0919


1.0918548887741395

In [7]:
#finding the average rating for each user
avg_rating_user = df.groupby("reviewerID")["overall","reviewerID"].mean().sort_values("overall")
avg_rating_user

<ipython-input-7-ab11ddabe163>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  avg_rating_user = df.groupby("reviewerID")["overall","reviewerID"].mean().sort_values("overall")


,overall
reviewerID,
A1W522Z24EPBJB,1.0
A2DPSPXFJ507C0,1.0
A1GQLVT0SWAWU,1.0
A1KLA02LZXAT46,1.0
A2MHHSACEJANSX,1.0
...,...
A15QGN6UXJVW9G,5.0
ANOJX4RAUJ9HL,5.0
A2RJT3IE2T6KXJ,5.0


In [10]:
#Create Series of high rating users 
high_rating_user = avg_rating_user[avg_rating_user["overall"]==5.0]
high_rating_user

,overall
reviewerID,
A2FINIRQNXOTI,5.0
ATWS89FH6Y6S4,5.0
A16Q479PYT0G6N,5.0
A3OKW5VRXZG3OQ,5.0
A3O9Q3154FPZLL,5.0
...,...
A15QGN6UXJVW9G,5.0
ANOJX4RAUJ9HL,5.0
A2RJT3IE2T6KXJ,5.0


In [15]:
#Create a list of reviewer IDs that rate high
high_rating_user_ids = list(high_rating_user.index)
len(high_rating_user_ids)


2822

In [8]:
#create new series of users who consistently rate 1 
low_rating_user = avg_rating_user[avg_rating_user["overall"]==1.0]
low_rating_user

,overall
reviewerID,
A1W522Z24EPBJB,1.0
A2DPSPXFJ507C0,1.0
A1GQLVT0SWAWU,1.0
A1KLA02LZXAT46,1.0
A2MHHSACEJANSX,1.0
A2RJTIE73NPN3C,1.0
ASWIC85F71H4J,1.0
A2TBE0N8JN6H4K,1.0
A1GPPMHYM6SMEW,1.0


In [12]:
##Create a list of reviewer IDs that rate low
low_rating_user_ids = list(low_rating_user.index)
low_rating_user_ids

['A1W522Z24EPBJB',
 'A2DPSPXFJ507C0',
 'A1GQLVT0SWAWU',
 'A1KLA02LZXAT46',
 'A2MHHSACEJANSX',
 'A2RJTIE73NPN3C',
 'ASWIC85F71H4J',
 'A2TBE0N8JN6H4K',
 'A1GPPMHYM6SMEW']

In [14]:
#Creating a clean dataframe without the high and low rating users

df_cleaned = df[~df.reviewerID.isin([high_rating_user_ids, low_rating_user_ids])]
df_cleaned

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1YJEY40YUW4SE,7806397051,Andrea,"[3, 4]",Very oily and creamy. Not at all what I expect...,1,Don't waste your money,1391040000,"01 30, 2014"
1,A60XNB876KYML,7806397051,Jessica H.,"[1, 1]",This palette was a decent price and I was look...,3,OK Palette!,1397779200,"04 18, 2014"
2,A3G6XNM240RMWA,7806397051,Karen,"[0, 1]",The texture of this concealer pallet is fantas...,4,great quality,1378425600,"09 6, 2013"
3,A1PQFP6SAJ6D80,7806397051,Norah,"[2, 2]",I really can't tell what exactly this thing is...,2,Do not work on my face,1386460800,"12 8, 2013"
4,A38FVHZTNQ271F,7806397051,Nova Amor,"[0, 0]","It was a little smaller than I expected, but t...",3,It's okay.,1382140800,"10 19, 2013"
...,...,...,...,...,...,...,...,...,...
198497,A2BLFCOPSMBOZ9,B00LLPT4HI,Dave Edmiston,"[0, 0]",Just a little dab of this shea butter should b...,5,A little dab...,1405468800,"07 16, 2014"
198498,A1UQBFCERIP7VJ,B00LLPT4HI,Margaret Picky,"[0, 0]",This shea butter is completely raw and unrefin...,5,Pure organic raw shea butter,1405296000,"07 14, 2014"
198499,A35Q0RBM3YNQNF,B00LLPT4HI,M. Hill,"[0, 0]",The skin is the body's largest organ and it ab...,5,One Pound Organic Grade A Unrefined Shea Butter,1405468800,"07 16, 2014"
198500,A3LGT6UZL99IW1,B00LLPT4HI,"Richard C. Drew ""Anaal Nathra/Uthe vas Bethod...","[0, 0]",I have very dry elbows and knees. I have a to...,5,This stuff is amazing!,1405382400,"07 15, 2014"


### Best Model 


In [7]:
#Import our meta data 
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')



In [36]:
meta_data_df = getDF("Data/meta_Beauty.json.gz")
meta_data_df

,asin,description,title,imUrl,salesRank,categories,price,related,brand
0,0205616461,"As we age, our once youthful, healthy skin suc...",Bio-Active Anti-Aging Serum (Firming Ultra-Hyd...,http://ecx.images-amazon.com/images/I/41DecrGO...,{'Health & Personal Care': 461765},"[[Beauty, Skin Care, Face, Creams & Moisturize...",NaN,NaN,NaN
1,0558925278,Mineral Powder Brush--Apply powder or mineral ...,Eco Friendly Ecotools Quality Natural Bamboo C...,http://ecx.images-amazon.com/images/I/51L%2BzY...,{'Beauty': 402875},"[[Beauty, Tools & Accessories, Makeup Brushes ...",NaN,NaN,NaN
2,0733001998,"From the Greek island of Chios, this Mastiha b...",Mastiha Body Lotion,http://ecx.images-amazon.com/images/I/311WK5y1...,{'Beauty': 540255},"[[Beauty, Skin Care, Body, Moisturizers, Lotio...",NaN,NaN,NaN
3,0737104473,Limited edition Hello Kitty Lipstick featuring...,Hello Kitty Lustre Lipstick (See sellers comme...,http://ecx.images-amazon.com/images/I/31u6Hrzk...,{'Beauty': 931125},"[[Beauty, Makeup, Lips, Lipstick]]",NaN,NaN,NaN
4,0762451459,"The mermaid is an elusive (okay, mythical) cre...",Stephanie Johnson Mermaid Round Snap Mirror,http://ecx.images-amazon.com/images/I/41y2%2BF...,NaN,"[[Beauty, Tools & Accessories, Mirrors, Makeup...",19.98,NaN,NaN
...,...,...,...,...,...,...,...,...,...
95,6041134473,Restore your skin's firmness and elasticity.\n...,Cellulite Massager Face Lift Face Massager 24k...,http://ecx.images-amazon.com/images/I/41Mv9hUf...,{'Beauty': 26646},"[[Beauty, Skin Care, Face]]",NaN,"{'also_bought': ['B00K7HKVQO', 'B00IODYRJ0', '...",NaN
96,6040652705,Cure time: 3 minutes. Performs with ease when ...,Atnails Nail Uv Gel - Extreme White - French M...,http://ecx.images-amazon.com/images/I/41EPV9ft...,{'Beauty': 554040},"[[Beauty, Makeup, Nails, Nail Polish]]",NaN,NaN,NaN
97,6041134511,Gold has magic energy of resisting oxidation.\...,24k Gold Vibrating Face Lifting Tightening T S...,http://ecx.images-amazon.com/images/I/511oshir...,{'Beauty': 397584},"[[Beauty, Skin Care, Sets & Kits]]",NaN,{'also_viewed': ['6041134473']},NaN
98,604113449X,The Extra 600 Titanium Micro Needles per rolle...,Derma Roller Titanium 1.0mm 600 Micro Needles ...,http://ecx.images-amazon.com/images/I/41kVoMvq...,{'Beauty': 80310},"[[Beauty, Skin Care, Face, Treatments & Masks]]",12.87,"{'also_bought': ['B008CEDY5O', 'B00C7DYBX0', '...",NaN


In [ ]:
meta_data_df.head(100)

In [49]:
best_model = SVD(n_factors=2, n_epochs=20, biased=True)
best_model.fit(trainset)

In [22]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1YJEY40YUW4SE,7806397051,Andrea,"[3, 4]",Very oily and creamy. Not at all what I expect...,1,Don't waste your money,1391040000,"01 30, 2014"
1,A60XNB876KYML,7806397051,Jessica H.,"[1, 1]",This palette was a decent price and I was look...,3,OK Palette!,1397779200,"04 18, 2014"
2,A3G6XNM240RMWA,7806397051,Karen,"[0, 1]",The texture of this concealer pallet is fantas...,4,great quality,1378425600,"09 6, 2013"
3,A1PQFP6SAJ6D80,7806397051,Norah,"[2, 2]",I really can't tell what exactly this thing is...,2,Do not work on my face,1386460800,"12 8, 2013"
4,A38FVHZTNQ271F,7806397051,Nova Amor,"[0, 0]","It was a little smaller than I expected, but t...",3,It's okay.,1382140800,"10 19, 2013"


In [99]:
## Subset data frame to show reviewers the products they have rated 
df_prior_ratings = pd.DataFrame(df.set_index("reviewerID"))
df_prior_ratings.drop(columns= ["reviewerName", "helpful", "reviewText", "overall", "summary", "unixReviewTime", "reviewTime"], inplace=True)
df_prior_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 198502 entries, A1YJEY40YUW4SE to A3UJRNI8UR4871
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   asin    198502 non-null  object
dtypes: object(1)
memory usage: 3.0+ MB


In [100]:
df_prior_ratings

,asin
reviewerID,
A1YJEY40YUW4SE,7806397051
A60XNB876KYML,7806397051
A3G6XNM240RMWA,7806397051
A1PQFP6SAJ6D80,7806397051
A38FVHZTNQ271F,7806397051
...,...
A2BLFCOPSMBOZ9,B00LLPT4HI
A1UQBFCERIP7VJ,B00LLPT4HI
A35Q0RBM3YNQNF,B00LLPT4HI


In [101]:
df_prior_ratings.loc["ALNFHVS3SC4FV", "asin"]

reviewerID
ALNFHVS3SC4FV    B00008BNZ4
ALNFHVS3SC4FV    B000AO2NXS
ALNFHVS3SC4FV    B000E0K7NQ
ALNFHVS3SC4FV    B000GUZ8HY
ALNFHVS3SC4FV    B000H8UN3Y
                    ...    
ALNFHVS3SC4FV    B00HVSQ4UE
ALNFHVS3SC4FV    B00IDWP4IA
ALNFHVS3SC4FV    B00IT1HKV4
ALNFHVS3SC4FV    B00KCFAZTE
ALNFHVS3SC4FV    B00KCTER3U
Name: asin, Length: 192, dtype: object

In [108]:
already_reviewed

['7806397051',
 'B00027D8IC',
 'B002PMLGOU',
 'B0030HKJ8I',
 'B004Z40048',
 'B00BN1MPPS']

In [112]:
len(meta_data_df["asin"])-6 == len(not_reviewed)

True

In [111]:
len(not_reviewed)

259198

In [107]:
already_reviewed=list(df_prior_ratings.loc["A1PQFP6SAJ6D80", "asin"])
not_reviewed = meta_data_df.copy()
not_reviewed = not_reviewed[not_reviewed.asin.isin(already_reviewed) == False]
#not_reviewed.drop(already_reviewed, inplace=True, axis=0)
not_reviewed

,asin,description,title,imUrl,salesRank,categories,price,related,brand
0,0205616461,"As we age, our once youthful, healthy skin suc...",Bio-Active Anti-Aging Serum (Firming Ultra-Hyd...,http://ecx.images-amazon.com/images/I/41DecrGO...,{'Health & Personal Care': 461765},"[[Beauty, Skin Care, Face, Creams & Moisturize...",NaN,NaN,NaN
1,0558925278,Mineral Powder Brush--Apply powder or mineral ...,Eco Friendly Ecotools Quality Natural Bamboo C...,http://ecx.images-amazon.com/images/I/51L%2BzY...,{'Beauty': 402875},"[[Beauty, Tools & Accessories, Makeup Brushes ...",NaN,NaN,NaN
2,0733001998,"From the Greek island of Chios, this Mastiha b...",Mastiha Body Lotion,http://ecx.images-amazon.com/images/I/311WK5y1...,{'Beauty': 540255},"[[Beauty, Skin Care, Body, Moisturizers, Lotio...",NaN,NaN,NaN
3,0737104473,Limited edition Hello Kitty Lipstick featuring...,Hello Kitty Lustre Lipstick (See sellers comme...,http://ecx.images-amazon.com/images/I/31u6Hrzk...,{'Beauty': 931125},"[[Beauty, Makeup, Lips, Lipstick]]",NaN,NaN,NaN
4,0762451459,"The mermaid is an elusive (okay, mythical) cre...",Stephanie Johnson Mermaid Round Snap Mirror,http://ecx.images-amazon.com/images/I/41y2%2BF...,NaN,"[[Beauty, Tools & Accessories, Mirrors, Makeup...",19.98,NaN,NaN
...,...,...,...,...,...,...,...,...,...
259199,B00LP2YB8E,Color: White\nFullness72 inches\nCenter Gather...,2t 2t Edge Crystal Rhinestones Bridal Wedding ...,http://ecx.images-amazon.com/images/I/41E630m-...,NaN,"[[Beauty, Hair Care, Styling Tools, Styling Ac...",NaN,NaN,NaN
259200,B00LOS7MEE,"The secret to long lasting colors, healthy nai...",French Manicure Gel Nail Polish Set - &quot;Se...,http://ecx.images-amazon.com/images/I/41skHL1O...,{'Beauty': 108820},"[[Beauty, Makeup, Nails, Nail Polish]]",NaN,"{'also_viewed': ['B0057JCYYE', 'B00LMXHR1Y', '...",NaN
259201,B00LPVG6V0,ResQ Organics Face & Body Wash - With Aloe Ver...,ResQ Organics Face &amp; Body Wash - Aloe Vera...,http://ecx.images-amazon.com/images/I/31C1w4Ku...,NaN,"[[Beauty, Skin Care, Face, Creams & Moisturize...",NaN,NaN,NaN
259202,B00LTDUHJQ,Color: White\n2 Tier \nFullness 72 inches\nSew...,2 Tier Tulle Elbow Wedding Veil with Ribbon Ed...,http://ecx.images-amazon.com/images/I/51%2B%2B...,NaN,"[[Beauty, Hair Care, Styling Tools, Styling Ac...",NaN,NaN,NaN


In [73]:
already_reviewed

['B001U9M2EW', 'B0027UUDFS', 'B003X3S26U', 'B005GHP5UC', 'B00BGCX63O']

In [78]:
meta_data_df[meta_data_df["asin"]=="B001U9M2EW"]

,asin,description,title,imUrl,salesRank,categories,price,related,brand
72209,B001U9M2EW,"Leave in, daily hair treatment that's super en...","Agadir Argan Oil Hair Treatment, 4-Ounce",http://ecx.images-amazon.com/images/I/41X0Ds-9...,{'Beauty': 140},"[[Beauty, Hair Care, Hair & Scalp Treatments]]",15.6,"{'also_bought': ['B00DULW2YS', 'B00DGDPEL8', '...",Agadir


In [70]:
meta_data_df.shape

(259204, 9)

In [72]:
df["asin"].value_counts()

B004OHQR1Q    431
B0043OYFKU    403
B0069FDR96    391
B000ZMBSPE    389
B00150LT40    329
             ... 
B006HTNOHY      5
B007WNVYW6      5
B005C78B9I      5
B000VHA9YO      5
B00E52F6G2      5
Name: asin, Length: 12101, dtype: int64

In [ ]:
#Creating new data set  
#df_prior_ratings.to_csv('Data/df_prior_ratings.csv')

In [113]:
#Function 

def buyer_recommended_products(): 
    
    buyer = input("reviewerID: ")
    n_recs = int(input("How many recommendations? "))
    
    already_reviewed = list(df_prior_ratings.loc[buyer, "asin"])
    not_reviewed = meta_data_df.copy()
    not_reviewed = not_reviewed[not_reviewed.asin.isin(already_reviewed) == False]
    #not_reviewed.drop(already_reviewed, inplace=True)
    not_reviewed.reset_index(inplace=True)
    not_reviewed["predicted_rating"] = not_reviewed["asin"].apply(lambda x: best_model.predict(buyer, x).est)
    not_reviewed.sort_values(by="predicted_rating", ascending=False, inplace=True)
    return not_reviewed.head(n_recs)
    

In [115]:
buyer_recommended_products()

reviewerID: A3NLEWJ1A8CF3P
How many recommendations? 20
['B001U9M2EW', 'B0027UUDFS', 'B003X3S26U', 'B005GHP5UC', 'B00BGCX63O']


,index,asin,description,title,imUrl,salesRank,categories,price,related,brand,predicted_rating
114807,114810,B0045W22SW,Unique Feature: Dense and rounded head\nFuncti...,Sigma F82 - Round Kabuki TM,http://ecx.images-amazon.com/images/I/3178N02L...,{'Beauty': 11302},"[[Beauty, Tools & Accessories, Makeup Brushes ...",24.00,"{'also_bought': ['B003UH0528', 'B0045W4X16', '...",Sigma Beauty,5.000000
131404,131407,B004TSF8R4,Designed with pro makeup artist Samantha Chapm...,real Techniques Powder Brush,http://ecx.images-amazon.com/images/I/31KkPQeb...,{'Beauty': 572},"[[Beauty, Tools & Accessories, Makeup Brushes ...",9.95,"{'also_bought': ['B00L74HOCK', 'B00L72DORG', '...",Real Techniques,5.000000
200567,200571,B00A1Y177A,A linear holographic nail polish that will tak...,"Color Club Halographic Hues Nail Polish, Light...",http://ecx.images-amazon.com/images/I/51J8BWjs...,{'Beauty': 132334},"[[Beauty, Makeup, Nails, Nail Polish]]",8.00,"{'also_bought': ['B00AO7AOOG', 'B00GHNOQ54', '...",Color Club,5.000000
54683,54683,B001B1T2SS,"Highlights:For two-handle kitchen, tub and lav...",China Glaze Polish Fairy Dust 70563/551 Discon...,http://ecx.images-amazon.com/images/I/41cXQNa9...,{'Beauty': 19626},"[[Beauty, Makeup, Nails, Nail Polish]]",4.73,"{'also_bought': ['B002SICH1C', 'B008A3WGTM', '...",Delta,4.970765
75524,75526,B0027VCNGO,Kit Includes 1/2 and 4oz Refill,Seche Vite Dry Fast Top Coat High Shine Profes...,http://ecx.images-amazon.com/images/I/51H6ErKy...,{'Beauty': 3413},"[[Beauty, Tools & Accessories, Nail Tools, Set...",14.92,"{'also_bought': ['B0016A8DT8', 'B000QUI920', '...",Seche Vite,4.965846
40690,40690,B000XU49MW,Easy to clean and flatters any bathroom d&#xFF...,"Danielle Square Acrylic Cotton Ball Holder, Si...",http://ecx.images-amazon.com/images/I/41GmQI7X...,{'Beauty': 966},"[[Beauty, Tools & Accessories, Bags & Cases, R...",5.98,"{'also_bought': ['B000XU9UJE', 'B00DR2FXAA', '...",Danielle Enterprises,4.956796
69684,69684,B001QFDTWO,Specially formulated for all over body moistur...,Gold Bond Ultimate Healing Skin Therapy Lotio...,http://ecx.images-amazon.com/images/I/31MVnx8n...,{'Health & Personal Care': 45308},"[[Beauty, Skin Care, Body, Moisturizers, Lotio...",9.00,"{'also_bought': ['B008B9L6WS', 'B00IWQY8SE', '...",Gold Bond,4.955561
146085,146089,B005HO0A18,,"Dove Body Wash with NutriumMoisture, Deep Mois...",http://ecx.images-amazon.com/images/I/41G1XnJT...,{'Beauty': 66},"[[Beauty, Bath & Body, Cleansers, Body Washes]]",21.99,"{'also_bought': ['B007UZNS5W', 'B00BGN8PLG', '...",Dove,4.951093
11247,11247,B000A409J8,L'Oreal Quick Blue Powder Bleach 1 Lb is a hig...,L'Oreal Quick Blue Powder Bleach 1 Lb,http://ecx.images-amazon.com/images/I/41asGN71...,{'Beauty': 1858},"[[Beauty, Hair Care, Hair & Scalp Treatments]]",17.52,"{'also_bought': ['B003Y6182G', 'B003Y5VLX8', '...",L&#39;Oreal Paris,4.944059
89982,89984,B002ZNJJLQ,Deluxe cosmetic organizer with drawer.,Rucci Cosmetic Organizer with Drawer,http://ecx.images-amazon.com/images/I/3100K0F6...,{'Health & Personal Care': 57803},"[[Beauty, Tools & Accessories, Bags & Cases, T...",NaN,"{'also_bought': ['B004KKXA58', 'B00ICB1P2G', '...",NaN,4.941688


In [65]:
buyer_recommended_products()

reviewerID: A3NLEWJ1A8CF3P
How many recommendations? 25


,index,asin,description,title,imUrl,salesRank,categories,price,related,brand,predicted_rating
200571,200571,B00A1Y177A,A linear holographic nail polish that will tak...,"Color Club Halographic Hues Nail Polish, Light...",http://ecx.images-amazon.com/images/I/51J8BWjs...,{'Beauty': 132334},"[[Beauty, Makeup, Nails, Nail Polish]]",8.00,"{'also_bought': ['B00AO7AOOG', 'B00GHNOQ54', '...",Color Club,5.000000
131407,131407,B004TSF8R4,Designed with pro makeup artist Samantha Chapm...,real Techniques Powder Brush,http://ecx.images-amazon.com/images/I/31KkPQeb...,{'Beauty': 572},"[[Beauty, Tools & Accessories, Makeup Brushes ...",9.95,"{'also_bought': ['B00L74HOCK', 'B00L72DORG', '...",Real Techniques,5.000000
114810,114810,B0045W22SW,Unique Feature: Dense and rounded head\nFuncti...,Sigma F82 - Round Kabuki TM,http://ecx.images-amazon.com/images/I/3178N02L...,{'Beauty': 11302},"[[Beauty, Tools & Accessories, Makeup Brushes ...",24.00,"{'also_bought': ['B003UH0528', 'B0045W4X16', '...",Sigma Beauty,5.000000
54683,54683,B001B1T2SS,"Highlights:For two-handle kitchen, tub and lav...",China Glaze Polish Fairy Dust 70563/551 Discon...,http://ecx.images-amazon.com/images/I/41cXQNa9...,{'Beauty': 19626},"[[Beauty, Makeup, Nails, Nail Polish]]",4.73,"{'also_bought': ['B002SICH1C', 'B008A3WGTM', '...",Delta,4.970765
75526,75526,B0027VCNGO,Kit Includes 1/2 and 4oz Refill,Seche Vite Dry Fast Top Coat High Shine Profes...,http://ecx.images-amazon.com/images/I/51H6ErKy...,{'Beauty': 3413},"[[Beauty, Tools & Accessories, Nail Tools, Set...",14.92,"{'also_bought': ['B0016A8DT8', 'B000QUI920', '...",Seche Vite,4.965846
40690,40690,B000XU49MW,Easy to clean and flatters any bathroom d&#xFF...,"Danielle Square Acrylic Cotton Ball Holder, Si...",http://ecx.images-amazon.com/images/I/41GmQI7X...,{'Beauty': 966},"[[Beauty, Tools & Accessories, Bags & Cases, R...",5.98,"{'also_bought': ['B000XU9UJE', 'B00DR2FXAA', '...",Danielle Enterprises,4.956796
69684,69684,B001QFDTWO,Specially formulated for all over body moistur...,Gold Bond Ultimate Healing Skin Therapy Lotio...,http://ecx.images-amazon.com/images/I/31MVnx8n...,{'Health & Personal Care': 45308},"[[Beauty, Skin Care, Body, Moisturizers, Lotio...",9.00,"{'also_bought': ['B008B9L6WS', 'B00IWQY8SE', '...",Gold Bond,4.955561
146089,146089,B005HO0A18,,"Dove Body Wash with NutriumMoisture, Deep Mois...",http://ecx.images-amazon.com/images/I/41G1XnJT...,{'Beauty': 66},"[[Beauty, Bath & Body, Cleansers, Body Washes]]",21.99,"{'also_bought': ['B007UZNS5W', 'B00BGN8PLG', '...",Dove,4.951093
11247,11247,B000A409J8,L'Oreal Quick Blue Powder Bleach 1 Lb is a hig...,L'Oreal Quick Blue Powder Bleach 1 Lb,http://ecx.images-amazon.com/images/I/41asGN71...,{'Beauty': 1858},"[[Beauty, Hair Care, Hair & Scalp Treatments]]",17.52,"{'also_bought': ['B003Y6182G', 'B003Y5VLX8', '...",L&#39;Oreal Paris,4.944059
89984,89984,B002ZNJJLQ,Deluxe cosmetic organizer with drawer.,Rucci Cosmetic Organizer with Drawer,http://ecx.images-amazon.com/images/I/3100K0F6...,{'Health & Personal Care': 57803},"[[Beauty, Tools & Accessories, Bags & Cases, T...",NaN,"{'also_bought': ['B004KKXA58', 'B00ICB1P2G', '...",NaN,4.941688


In [66]:
df[df["reviewerID"]=="A3NLEWJ1A8CF3P"]

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
66082,A3NLEWJ1A8CF3P,B001U9M2EW,Tony B.,"[1, 1]",This is one of the best leave-in treatments on...,5,One of the best leave-in treatments,1329955200,"02 23, 2012"
68539,A3NLEWJ1A8CF3P,B0027UUDFS,Tony B.,"[0, 0]",This soap is incredible - I have very oily ski...,5,Excellent for oily skin,1329955200,"02 23, 2012"
97612,A3NLEWJ1A8CF3P,B003X3S26U,Tony B.,"[1, 2]",This is by far one of the best hair masks for ...,5,One of the best,1329955200,"02 23, 2012"
124685,A3NLEWJ1A8CF3P,B005GHP5UC,Tony B.,"[2, 2]",This is definitely a good product for skin and...,3,Good but not as leave-in treatment,1329955200,"02 23, 2012"
177873,A3NLEWJ1A8CF3P,B00BGCX63O,Tony B.,"[0, 0]",My wife's been using thermal water based produ...,5,Great product,1400716800,"05 22, 2014"


In [85]:
df_asin_check = pd.DataFrame(df["asin"].unique(), columns=["recommendation_asins"])

In [86]:
df_asin_check

,recommendation_asins
0,7806397051
1,9759091062
2,9788072216
3,9790790961
4,9790794231
...,...
12096,B00L5KTZ0K
12097,B00L6Q3BH6
12098,B00LCEROA2
12099,B00LG63DOM


In [88]:
unique_meta_data_asin = list(meta_data_df["asin"].unique())


In [92]:
df_asin_check["is_in_meta"] = df_asin_check["recommendation_asins"].isin(unique_meta_data_asin)

In [98]:
df_asin_check["is_in_meta"].value_counts()

True    12101
Name: is_in_meta, dtype: int64